In [1]:
import numpy as np
import pandas as pd
import os, sys, time, zipfile
import tensorflow as tf

bert_path = '../../dataset/bert/'
sys.path.append(os.path.abspath(os.path.dirname(bert_path)))
import modeling
import extract_features
import tokenization

In [ ]:
read_path = '../../dataset/origin/'


In [6]:
'''
we feed BERT the data from these three files.
For each line, we want to obtain contextual embeddings for the 3 target words (A, B, Pronoun).
Here are some helper functions to keep track of the offsets of the target words.
'''
def compute_offset_no_spaces(text, offset):
    count = 0
    for pos in range(offset):
        if text[pos] != " ": count +=1
    return count

def count_chars_no_special(text):
    count = 0
    special_char_list = ["#"]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count+=1
    return count

def count_length_no_special(text):
    count=0
    special_char_list = ["#", " "]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: count+=1
    return count

In [ ]:
'''
the data from a file passes it through BERT to obtain contextual embeddings for the target words,
then returns these embeddings in the emb DataFrame.
Below, we will use it 3 times,
once for each of the files gap-test, gap-development, gap-validation
'''
path = '../../dataset/bert/'
def run_bert(data):
    text = data['Text']
    text.to_csv('input.txt', index=False, header=False)
    os.system("python extract_features.py\
    --input_file=input.txt\
    --output_file=output.jsonl\
    --vocab_file="+path+"vocab.txt\
    --bert_config_file="+path+"bert_config.json\
    --init_checkpoint="+path+"bert_model.ckpt\
    --layers=-1\
    --max_seq_length=256\
    --batch_size=8")
    bert_output = pd.read_json("output.jsonl", lines=True)
    os.system("rm output.jsonl")
    os.system("rm input.txt")
    index = data.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index=index, columns=columns)
    emb.index.name = "ID"
    for i in range(len(data)):
        P = data.loc[i,"Pronoun"].lower()
        A = data.loc[i, "A"].lower()
        B = data.loc[i, "B"].lower()
        
        P_offset = compute_offset_no_spaces(data.loc[i, "Text"], data.loc[i, "Pronoun-offset"])
        A_offset = compute_offset_no_spaces(data.loc[i, "Text"], data.loc[i, "A-offset"])